In [11]:
import time
import re
from pyspark.sql import SparkSession, functions as sf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

file_path = 'data/sample-2mb-text-file.txt'
spark = SparkSession.builder.appName("WordCountExample").getOrCreate()


# Démonstration du Catalyst Optimizer

## Standard Python

In [12]:
start = time.time()

with open(file_path) as file:
    text = file.read()

words = re.split("\s+", text)
result = []
for word in set(words):
    result.append((word, words.count(word)))

end = time.time()

print("Word count :", result)
print("Elasped time :", end - start)

Word count : [('Ultricies', 226), ('morbi', 2293), ('condimentum', 826), ('Libero', 170), ('Facilisi', 145), ('interdum', 1024), ('fusce', 385), ('pharetra', 1632), ('Sagittis', 245), ('Nullam', 148), ('Elementum', 328), ('Tellus', 430), ('maecenas.', 189), ('suspendisse', 1176), ('Potenti', 38), ('justo', 768), ('lorem.', 221), ('cras.', 253), ('quam', 1804), ('est.', 301), ('Suscipit', 55), ('penatibus.', 24), ('ac', 2573), ('tincidunt', 2277), ('erat.', 118), ('nam.', 104), ('hendrerit.', 103), ('auctor', 968), ('aliquam.', 387), ('natoque', 210), ('nullam.', 141), ('eleifend.', 108), ('amet', 4594), ('curabitur.', 69), ('netus', 630), ('lacinia.', 63), ('odio.', 329), ('vehicula', 180), ('velit.', 286), ('Cursus', 314), ('justo.', 136), ('elementum.', 344), ('commodo', 1336), ('ac.', 506), ('ridiculus', 191), ('a.', 387), ('nulla.', 427), ('dolor.', 223), ('orci', 1594), ('dis.', 39), ('Magna', 197), ('ipsum', 1430), ('vel.', 367), ('Eu', 531), ('habitasse.', 64), ('eleifend', 598)

## Spark with RDD

In [13]:
start = time.time()

text = spark.sparkContext.textFile(file_path)
word_counts_rdd = text.flatMap(lambda line: line.split("\s+")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b)
result = word_counts_rdd.collect()

end = time.time()
print("Word count :", result)
print("Elasped time :", end - start)


## Using Spark with DataFrames

In [ ]:
start = time.time()

text = spark.read.text(file_path)
wordCounts = text.select(sf.explode(sf.split(text.value, "\s+")).alias("word")).groupBy("word").count()
result = wordCounts.collect()

end = time.time()
print("Word count :", result)
print("Elasped time :", end - start)




Word count : [Row(word='Sit', count=867), Row(word='Elit', count=308), Row(word='vehicula.', count=39), Row(word='eros.', count=78), Row(word='nam.', count=104), Row(word='dui.', count=260), Row(word='porttitor', count=962), Row(word='eleifend.', count=108), Row(word='consectetur.', count=256), Row(word='senectus.', count=106), Row(word='odio', count=1815), Row(word='Turpis', count=373), Row(word='Sed', count=986), Row(word='volutpat', count=1923), Row(word='integer.', count=203), Row(word='interdum', count=1024), Row(word='pretium', count=1347), Row(word='Eu', count=531), Row(word='semper.', count=239), Row(word='nisl.', count=295), Row(word='non.', count=433), Row(word='sem.', count=181), Row(word='ipsum.', count=233), Row(word='habitant.', count=104), Row(word='vel.', count=367), Row(word='volutpat.', count=342), Row(word='Sem', count=199), Row(word='Quisque', count=152), Row(word='Cursus', count=314), Row(word='tempor.', count=182), Row(word='sagittis', count=1460), Row(word='felis

On constate que le calcul est plus efficace en utilisant des DataFrames plutôt que les RDD.
Cela est dû au fait que les DataFrames exploitent le Catalyst Optimizer

# Démonstration de l'optimisation des requêtes

In [ ]:
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

data = [("Alice", 25), ("Bob", 18), ("Charlie", 30), ("David", 22), ("Eva", 28)]

df = spark.createDataFrame(data, schema=schema)

## Version non optimisée

In [ ]:
start = time.time()

df_non_optimized = df.select("*").filter(df["age"] > 21)
result = df_non_optimized.collect()

end = time.time()
print("Word count :", result)
print("Elasped time :", end - start)

Py4JJavaError: An error occurred while calling o140.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 7.0 failed 1 times, most recent failure: Lost task 6.0 in stage 7.0 (TID 12) (DESKTOP-6D4U070 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCh

## Version optimisée

In [ ]:
start = time.time()

df_optimized  = df.select("*").filter(df["age"] > 21)
result = df_optimized.collect()

end = time.time()
print("Word count :", result)
print("Elasped time :", end - start)

On constate que la version optimisée est bien plus rapide

In [ ]:
spark.stop()